<a href="https://colab.research.google.com/github/carmea2025-dev/conversor-colab/blob/main/Conversor_BPA_Com_Botoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# @title Projeto BAHIA QUE PRODUZ E ALIMENTA - Plano de Ação { display-mode: "form" }

from IPython.display import display, HTML, clear_output
import pandas as pd
import io
import requests
from google.colab import files
import ipywidgets as widgets
from openpyxl import load_workbook

# Exibe o título acima da interface, visível mesmo com o código oculto
display(HTML("""
<h2 style="text-align:center; color:#2c3e50; font-family:'Trebuchet MS', sans-serif;">
📘 Projeto <b>BAHIA QUE PRODUZ E ALIMENTA</b> - Plano de Ação
</h2>
<hr style="border:1px solid #ccc;">
"""))

# URLs dos arquivos no GitHub (raw)
url_mapeamento = "https://raw.githubusercontent.com/carmea2025-dev/conversor-colab/main/Mapeamento_Colunas.xlsx"
url_modelo = "https://raw.githubusercontent.com/carmea2025-dev/conversor-colab/main/PLANO%20DE%20AÇÃO%20-%20BPA%20-%20FINAL.xlsx"

def baixar_arquivo(url, nome_arquivo):
    r = requests.get(url)
    if r.status_code == 200:
        with open(nome_arquivo, "wb") as f:
            f.write(r.content)
        print(f"✅ {nome_arquivo} baixado com sucesso.")
    else:
        print(f"❌ Erro ao baixar {nome_arquivo}. Verifique o link do GitHub.")

# Baixa os arquivos do GitHub automaticamente
baixar_arquivo(url_mapeamento, "Mapeamento_Colunas.xlsx")
baixar_arquivo(url_modelo, "PLANO_DE_ACAO_BPA_FINAL.xlsx")

def converter_arquivo(uploaded_file):
    try:
        # Carrega mapeamento e modelo
        mapeamento = pd.read_excel("Mapeamento_Colunas.xlsx", engine='openpyxl')
        modelo_df = pd.read_excel("PLANO_DE_ACAO_BPA_FINAL.xlsx", engine='openpyxl')

        nome_arquivo = list(uploaded_file.keys())[0]
        conteudo = uploaded_file[nome_arquivo]['content']

        # Detecta extensão e lê o arquivo BPA
        if nome_arquivo.endswith(".xls"):
            df_bpa = pd.read_excel(io.BytesIO(conteudo), engine='xlrd')
        elif nome_arquivo.endswith(".xlsx"):
            df_bpa = pd.read_excel(io.BytesIO(conteudo), engine='openpyxl')
        else:
            print("⚠️ Formato de arquivo não suportado. Envie .xls ou .xlsx.")
            return

        # Renomeia colunas com base no mapeamento
        col_map = dict(zip(mapeamento['Coluna_Origem'], mapeamento['Coluna_Destino']))
        df_convertido = df_bpa.rename(columns=col_map)

        # Carrega o modelo com formatação (openpyxl)
        wb = load_workbook("PLANO_DE_ACAO_BPA_FINAL.xlsx")
        ws = wb.active

        # Cria uma lista das colunas do modelo
        colunas_modelo = list(modelo_df.columns)

        # Atualiza células mantendo formatação
        for col_idx, col_name in enumerate(colunas_modelo, start=1):
            if col_name in df_convertido.columns:
                valores = df_convertido[col_name].tolist()
                for row_idx, valor in enumerate(valores, start=2):  # começa na linha 2
                    ws.cell(row=row_idx, column=col_idx, value=valor)

        # Salva o arquivo final com a formatação preservada
        nome_saida = "PLANO DE AÇÃO - BPA - CONVERTIDO.xlsx"
        wb.save(nome_saida)
        print("✅ Conversão concluída com sucesso! Formatação preservada.")
        files.download(nome_saida)

    except Exception as e:
        print(f"❌ Erro durante a conversão: {e}")

def interface():
    upload_button = widgets.FileUpload(accept=".xls,.xlsx", multiple=False)
    convert_button = widgets.Button(description="Converter e Baixar", button_style="success")

    def ao_clicar(_):
        clear_output(wait=True)
        display(HTML("""
        <h2 style="text-align:center; color:#2c3e50; font-family:'Trebuchet MS', sans-serif;">
        📘 Projeto <b>BAHIA QUE PRODUZ E ALIMENTA</b> - Plano de Ação
        </h2>
        <hr style="border:1px solid #ccc;">
        """))
        display(upload_button, convert_button)
        if upload_button.value:
            converter_arquivo(upload_button.value)
        else:
            print("⚠️ Envie um arquivo BPA primeiro.")

    convert_button.on_click(ao_clicar)
    display(upload_button, convert_button)

# Exibe a interface no Colab
interface()


FileUpload(value={'PLANO DE AÇÃO - BPA.xlsx': {'metadata': {'name': 'PLANO DE AÇÃO - BPA.xlsx', 'type': 'appli…

Button(button_style='success', description='Converter e Baixar', style=ButtonStyle())

✅ Conversão concluída com sucesso! Formatação preservada.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>